## 인공지능과제 14
Decision tree 필요 단계
0. 데이터 전처리
    - feature 중 float 가 아닌 데이터에 대해 전처리 진행.
    - name -> company 추출, power 등 단위가 있는 feature에 대해, 단위 제거 후 float 로 변환.
1. 데이터 분석 후, DT에 맞게 전처리 진행.
2. 기본적인 Decision tree 모델링
    - 하이퍼파라미터(max_depth 등 )
    - GridSearchCV 등을 통해 파라미터 튜닝 진행.


In [208]:
from __future__ import print_function
import os

from io import StringIO
from IPython.display import Image, display

from sklearn.tree import export_graphviz
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [209]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [210]:
import pandas as pd
import numpy as np

In [211]:
data = pd.read_csv('train.csv')

In [212]:
data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


In [213]:
print(data.dtypes)

Name                  object
Location              object
Year                   int64
Kilometers_Driven      int64
Fuel_Type             object
Transmission          object
Owner_Type            object
Mileage               object
Engine                object
Power                 object
Seats                float64
New_Price             object
Price                float64
dtype: object


In [214]:
train, test = train_test_split(data, test_size=0.3, random_state=42)
train_data = train.drop(labels="New_Price", axis = 1)
test = test.drop(labels="New_Price", axis = 1)

In [215]:
train_data.describe()

,Year,Kilometers_Driven,Seats,Price
count,4213.000000,4213.000000,4185.000000,4213.000000
mean,2013.396392,57529.234987,5.277419,9.466622
std,3.237586,38099.559922,0.805531,11.229414
min,1998.000000,600.000000,0.000000,0.440000
25%,2011.000000,34000.000000,5.000000,3.500000
50%,2014.000000,52827.000000,5.000000,5.680000
75%,2016.000000,72100.000000,5.000000,9.900000
max,2019.000000,775000.000000,10.000000,160.000000


In [216]:
train_data = train_data[train_data['Mileage'].notna()]
print(train_data.shape)
train_data = train_data[train_data['Engine'].notna()]
print(train_data.shape)
train_data = train_data[train_data['Power'].notna()]
print(train_data.shape)
train_data = train_data[train_data['Seats'].notna()]
print(train_data.shape)

(4212, 12)
(4188, 12)
(4188, 12)
(4184, 12)


In [217]:
print(train_data['Location'].unique())
print(train_data['Fuel_Type'].unique())
print(train_data['Transmission'].unique())
print(train_data['Owner_Type'].unique())

['Jaipur' 'Hyderabad' 'Kochi' 'Kolkata' 'Ahmedabad' 'Mumbai' 'Coimbatore'
 'Chennai' 'Bangalore' 'Delhi' 'Pune']
['Diesel' 'Petrol' 'CNG' 'LPG']
['Manual' 'Automatic']
['First' 'Second' 'Third' 'Fourth & Above']


In [218]:
train_data.isnull().sum()

Name                 0
Location             0
Year                 0
Kilometers_Driven    0
Fuel_Type            0
Transmission         0
Owner_Type           0
Mileage              0
Engine               0
Power                0
Seats                0
Price                0
dtype: int64

In [219]:
train_data = train_data.reset_index(drop=True)

In [220]:
for i in range(train_data.shape[0]):
    train_data.at[i, 'Company'] = train_data['Name'][i].split()[0]
    train_data.at[i, 'Mileage(km/kg)'] = train_data['Mileage'][i].split()[0]
    train_data.at[i, 'Engine(CC)'] = train_data['Engine'][i].split()[0]
    train_data.at[i, 'Power(bhp)'] = train_data['Power'][i].split()[0]


In [221]:
x = 'n'
count = 0
position = []
for i in range(train_data.shape[0]):
    if train_data['Power(bhp)'][i]=='null':
        x = 'Y'
        count = count + 1
        position.append(i)
print(x)
print(count)
print(position)

Y
66
[167, 201, 337, 376, 392, 448, 505, 554, 591, 600, 641, 692, 710, 752, 823, 879, 917, 920, 935, 1069, 1120, 1208, 1283, 1288, 1351, 1454, 1546, 1738, 1821, 1835, 1837, 1938, 2087, 2089, 2133, 2140, 2219, 2249, 2257, 2271, 2315, 2425, 2509, 2551, 2648, 2704, 2713, 2826, 2845, 2979, 3046, 3103, 3141, 3151, 3158, 3189, 3203, 3208, 3354, 3570, 3605, 3924, 3933, 4004, 4064, 4067]


In [222]:
train_data = train_data.drop(train_data.index[position])
train_data = train_data.reset_index(drop=True)

In [223]:
train_data['Mileage(km/kg)'] = train_data['Mileage(km/kg)'].astype(float)
train_data['Engine(CC)'] = train_data['Engine(CC)'].astype(float)
train_data['Power(bhp)'] = train_data['Power(bhp)'].astype(float)
train_data['Year'] = train_data['Year'].astype(float)
train_data['Kilometers_Driven'] = train_data['Kilometers_Driven'].astype(float)


In [224]:
train_data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,Mileage(km/kg),Engine(CC),Power(bhp)
0,Volkswagen Polo Diesel Trendline 1.2L,Jaipur,2011.0,77000.0,Diesel,Manual,First,22.07 kmpl,1199 CC,73.9 bhp,5.0,2.90,Volkswagen,22.07,1199.0,73.9
1,Hyundai i10 Sportz,Hyderabad,2016.0,19947.0,Petrol,Manual,First,20.36 kmpl,1197 CC,78.9 bhp,5.0,4.50,Hyundai,20.36,1197.0,78.9
2,Mahindra XUV500 W8 2WD,Kochi,2014.0,70963.0,Diesel,Manual,First,15.1 kmpl,2179 CC,140 bhp,7.0,7.83,Mahindra,15.10,2179.0,140.0
3,Maruti Swift VDI BSIV,Kolkata,2013.0,115195.0,Diesel,Manual,First,25.2 kmpl,1248 CC,74 bhp,5.0,3.85,Maruti,25.20,1248.0,74.0
4,Volkswagen Polo Petrol Trendline 1.2L,Kochi,2011.0,58752.0,Petrol,Manual,First,16.47 kmpl,1198 CC,73.9 bhp,5.0,3.32,Volkswagen,16.47,1198.0,73.9


In [225]:
train_data.shape

(4118, 16)

In [226]:
train_data.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Company,Mileage(km/kg),Engine(CC),Power(bhp)
0,Volkswagen Polo Diesel Trendline 1.2L,Jaipur,2011.0,77000.0,Diesel,Manual,First,22.07 kmpl,1199 CC,73.9 bhp,5.0,2.90,Volkswagen,22.07,1199.0,73.9
1,Hyundai i10 Sportz,Hyderabad,2016.0,19947.0,Petrol,Manual,First,20.36 kmpl,1197 CC,78.9 bhp,5.0,4.50,Hyundai,20.36,1197.0,78.9
2,Mahindra XUV500 W8 2WD,Kochi,2014.0,70963.0,Diesel,Manual,First,15.1 kmpl,2179 CC,140 bhp,7.0,7.83,Mahindra,15.10,2179.0,140.0
3,Maruti Swift VDI BSIV,Kolkata,2013.0,115195.0,Diesel,Manual,First,25.2 kmpl,1248 CC,74 bhp,5.0,3.85,Maruti,25.20,1248.0,74.0
4,Volkswagen Polo Petrol Trendline 1.2L,Kochi,2011.0,58752.0,Petrol,Manual,First,16.47 kmpl,1198 CC,73.9 bhp,5.0,3.32,Volkswagen,16.47,1198.0,73.9


In [227]:
train_data.drop(["Name"],axis=1,inplace=True)
train_data.drop(["Mileage"],axis=1,inplace=True)
train_data.drop(["Engine"],axis=1,inplace=True)
train_data.drop(["Power"],axis=1,inplace=True)
train_data.drop(["Location"],axis=1,inplace=True)
train_data.drop(["Fuel_Type"],axis=1,inplace=True)
train_data.drop(["Transmission"],axis=1,inplace=True)
train_data.drop(["Owner_Type"],axis=1,inplace=True)
train_data.drop(["Company"],axis=1,inplace=True)
train_data["Price"] = train_data["Price"].astype(int)





In [228]:
train_data.head()

,Year,Kilometers_Driven,Seats,Price,Mileage(km/kg),Engine(CC),Power(bhp)
0,2011.0,77000.0,5.0,2,22.07,1199.0,73.9
1,2016.0,19947.0,5.0,4,20.36,1197.0,78.9
2,2014.0,70963.0,7.0,7,15.10,2179.0,140.0
3,2013.0,115195.0,5.0,3,25.20,1248.0,74.0
4,2011.0,58752.0,5.0,3,16.47,1198.0,73.9


In [229]:
train, test = train_test_split(train_data, random_state=42, test_size=0.3)


In [230]:
y_train = train.Price
train_x = train.drop(labels="Price", axis = 1)
y_test = test.Price
test_x = test.drop(labels="Price", axis = 1)

In [231]:
train_x.head()

,Year,Kilometers_Driven,Seats,Mileage(km/kg),Engine(CC),Power(bhp)
1588,2010.0,59196.0,5.0,15.60,1196.0,70.0
997,2014.0,97000.0,7.0,12.55,2982.0,169.0
3127,2016.0,46936.0,5.0,13.00,2143.0,204.0
1952,2016.0,26981.0,5.0,16.47,1198.0,73.9
1132,2013.0,38000.0,5.0,16.47,1198.0,73.9


In [232]:
y_train.head()

1588     2
997     17
3127    29
1952     6
1132     2
Name: Price, dtype: int64

In [233]:
train_x.shape
train_x.dtypes

Year                 float64
Kilometers_Driven    float64
Seats                float64
Mileage(km/kg)       float64
Engine(CC)           float64
Power(bhp)           float64
dtype: object

In [234]:
dt = DecisionTreeClassifier(random_state=42)
dt = dt.fit(train_x, y_train)

In [235]:
from sklearn.metrics import mean_squared_error

In [236]:
mean_squared_error(y_test,dt.predict(test_x))

33.80582524271845